In [1]:
# Data Processing
import numpy as np
from sklearn.preprocessing import LabelEncoder, normalize, StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

train_data = np.genfromtxt('trainData.csv', delimiter=',', dtype=str)
m = train_data.shape[0] - 1
d = train_data.shape[1]
print(m, d)
train_input_raw, train_output = train_data[1:,0:d-1].tolist(), train_data[1:,d-1].tolist()

24712 20


The above step uses `numpy`'s `genfromtxt` function since there are no missing values. Below I randomly print to get the actual types of data like textual information and specific fields

In [2]:
print([train_input_raw[0], train_output[0]])

[['28', '"admin."', '"single"', '"university.degree"', '"no"', '"yes"', '"no"', '"cellular"', '"aug"', '"mon"', '1', '999', '0', '"nonexistent"', '-2.9', '92.201', '-31.4', '0.861', '5076.2'], '1']


#### Preprocessing step 1: 
Remove quotes in textual information

#### Preprocessing step 2:
Convert numeric information to numbers as opposed to within single quotes. These fields are `0, 10, 11, 12, 14, ..., 19`.

In [3]:
for i in range(0, m-1):
    for j in range(0, d-1):
        if train_input_raw[i][j].find("\"") != -1:
            train_input_raw[i][j] = train_input_raw[i][j][1:-1] # removing double quotes in pairs
        else:
            train_input_raw[i][j] = float(train_input_raw[i][j])
    train_output[i] = float(train_output[i])
train_output = np.array(train_output).astype(float)

print(train_input_raw[0])
print(train_output[0])

[28.0, 'admin.', 'single', 'university.degree', 'no', 'yes', 'no', 'cellular', 'aug', 'mon', 1.0, 999.0, 0.0, 'nonexistent', -2.9, 92.201, -31.4, 0.861, 5076.2]
1.0


Now that the preprocessing has been completed, it is time to separate the input and outputs and begin training. Clearly, we might have to go with Ensemble methods since the data has both textual and numeric information in it.

In [4]:
rand_index = np.random.randint(low=0, high=m)
print(train_input_raw[rand_index])
print(train_output[rand_index])

[50.0, 'blue-collar', 'married', 'basic.4y', 'no', 'yes', 'no', 'cellular', 'jul', 'mon', 2.0, 999.0, 0.0, 'nonexistent', 1.4, 93.918, -42.7, 4.962, 5228.1]
1.0


Now that I have the data through a simple level of preprocessing, I am going to convert text data into numeric values. For this I am first going to tokenize and then merge this into the actual training input.

#### Preprocessing step 3:
Convert the text data into numbers using `LabelEncoder`.

#### Preprocessing step 4:
Normalize the data.

In [5]:
train_text_data = np.hstack((np.array(train_input_raw)[:,1:10], np.array(train_input_raw)[:,13:14]))
print('Size of training text data : {0}'.format(train_text_data.shape))

# Converting the textual data into one vector per training input
train_text_vectors = []

for j in range(0, train_text_data.shape[1]):
    lbl_enc = LabelEncoder()
    train_text_vectors.append(lbl_enc.fit_transform(train_text_data[:,j]))
train_text_vectors = np.array(train_text_vectors).T

print('Size of training text data: {0}'.format(train_text_vectors.shape))
print('Text: {0} --> Data: {1}'.format(train_text_data[0], train_text_vectors[0]))
print('Text: {0} --> Data: {1}'.format(train_text_data[10], train_text_vectors[10]))

Size of training text data : (24712, 10)
Size of training text data: (24712, 10)
Text: ['admin.' 'single' 'university.degree' 'no' 'yes' 'no' 'cellular' 'aug'
 'mon' 'nonexistent'] --> Data: [1 3 7 1 3 1 1 2 2 2]
Text: ['blue-collar' 'single' 'basic.9y' 'no' 'yes' 'no' 'cellular' 'jul' 'tue'
 'nonexistent'] --> Data: [2 3 3 1 3 1 1 4 4 2]


In [6]:
train_input = np.hstack((np.array(train_input_raw)[:,0:1], 
                         train_text_vectors[:,0:train_text_vectors.shape[1] - 1], 
                         np.array(train_input_raw)[:,10:13],
                         train_text_vectors[:,train_text_vectors.shape[1] - 1].reshape(-1, 1),
                         np.array(train_input_raw)[:,14:]
                        )).astype(float)
mean_reduce = StandardScaler()
train_input = mean_reduce.fit_transform(train_input)
print(train_input.shape)
print('Vector = {0}: \nNorm = {1}'.format(train_input[0], np.linalg.norm(train_input[0])))

train_input = normalize(train_input)
print(train_input.shape)
print('Vector = {0}: \nNorm = {1}'.format(train_input[0], np.linalg.norm(train_input[0])))

(24712, 19)
Vector = [-1.1518788  -1.04245572  1.36067936  1.05600834 -0.51164653  0.94134947
 -0.44949276 -0.75723938 -1.38470179 -0.70956021 -0.56425354  0.19653049
 -0.35249416  0.19500115 -1.90310477 -2.38261316  1.97762226 -1.5909553
 -1.25826956]: 
Norm = 5.250495168763574
(24712, 19)
Vector = [-0.21938479 -0.19854427  0.25915258  0.20112548 -0.09744729  0.17928775
 -0.08560959 -0.14422247 -0.26372785 -0.13514158 -0.10746673  0.03743085
 -0.06713541  0.03713957 -0.36246196 -0.45378828  0.37665443 -0.30301052
 -0.23964779]: 
Norm = 1.0


Now the preprocessing steps are completed. I will try my preprocessed dataset on multiple methods of classification.
For this, I will generate 10 folds (stratified) and use 10-fold cross validation to get the best method.

In [7]:
skf = StratifiedKFold(n_splits=10)
folds = []
for tr_split, va_split in skf.split(train_input, train_output):
    folds.append((tr_split, va_split))
for i in range(0, len(folds)):
    print("Fold {0}\tTrain number: {1} Validation number: {2}".format(i+1, len(folds[i][0]), len(folds[i][1])))

Fold 1	Train number: 22240 Validation number: 2472
Fold 2	Train number: 22240 Validation number: 2472
Fold 3	Train number: 22240 Validation number: 2472
Fold 4	Train number: 22240 Validation number: 2472
Fold 5	Train number: 22241 Validation number: 2471
Fold 6	Train number: 22241 Validation number: 2471
Fold 7	Train number: 22241 Validation number: 2471
Fold 8	Train number: 22241 Validation number: 2471
Fold 9	Train number: 22242 Validation number: 2470
Fold 10	Train number: 22242 Validation number: 2470


### Attempt 1: SVM with Gaussian Kernel with 10 fold cross validation

In [8]:
accu_train = prec_train = rcll_train = 0.0
accu_valid = prec_valid = rcll_valid = 0.0
g_svm = SVC(kernel='rbf', cache_size=2000)
for i in range(0, len(folds)):
    print("Performing fold {0}".format(i+1))
    g_svm.fit(train_input[folds[i][0]], train_output[folds[i][0]])

    train_pred = g_svm.predict(train_input[folds[i][0]])
    valid_pred = g_svm.predict(train_input[folds[i][1]])

    accu_train += accuracy_score(train_output[folds[i][0]], train_pred)
    prec_train += precision_score(train_output[folds[i][0]], train_pred)
    rcll_train += recall_score(train_output[folds[i][0]], train_pred)
    
    accu_valid += accuracy_score(train_output[folds[i][1]], valid_pred)
    prec_valid += precision_score(train_output[folds[i][1]], valid_pred)
    rcll_valid += recall_score(train_output[folds[i][1]], valid_pred)
    
print("Average training accuracy after 10 fold cross validation = {0}".format(accu_train/len(folds)))
print("Average training precision after 10 fold cross validation = {0}".format(prec_train/len(folds)))
print("Average training recall after 10 fold cross validation = {0}".format(rcll_train/len(folds)))

print("Average validation accuracy after 10 fold cross validation = {0}".format(accu_valid/len(folds)))
print("Average validation precision after 10 fold cross validation = {0}".format(prec_valid/len(folds)))
print("Average validation recall after 10 fold cross validation = {0}".format(rcll_valid/len(folds)))

Performing fold 1
Performing fold 2
Performing fold 3
Performing fold 4
Performing fold 5
Performing fold 6
Performing fold 7
Performing fold 8
Performing fold 9
Performing fold 10
Average training accuracy after 10 fold cross validation = 0.8958805422110278
Average training precision after 10 fold cross validation = 0.6147864168325933
Average training recall after 10 fold cross validation = 0.20294533040861612
Average validation accuracy after 10 fold cross validation = 0.8958804098640757
Average validation precision after 10 fold cross validation = 0.6136531230006101
Average validation recall after 10 fold cross validation = 0.20293958381681754


### Attempt 2: Random Forests with multiple number of trees and 10 fold cross validation

In [9]:
for m in range(3, 9):
    rnd_frst = RandomForestClassifier(n_estimators = m)
    accu_train = prec_train = rcll_train = 0.0
    accu_valid = prec_valid = rcll_valid = 0.0
    for i in range(0, len(folds)):
        rnd_frst.fit(train_input[folds[i][0]], train_output[folds[i][0]])

        train_pred = rnd_frst.predict(train_input[folds[i][0]])
        valid_pred = rnd_frst.predict(train_input[folds[i][1]])

        accu_train += accuracy_score(train_output[folds[i][0]], train_pred)
        prec_train += precision_score(train_output[folds[i][0]], train_pred)
        rcll_train += recall_score(train_output[folds[i][0]], train_pred)
    
        accu_valid += accuracy_score(train_output[folds[i][1]], valid_pred)
        prec_valid += precision_score(train_output[folds[i][1]], valid_pred)
        rcll_valid += recall_score(train_output[folds[i][1]], valid_pred)
    
    print("Average training accuracy after 10 fold cross validation = {0} for m = {1}".format(accu_train/len(folds), m))
    print("Average training precision after 10 fold cross validation = {0} for m = {1}".format(prec_train/len(folds), m))
    print("Average training recall after 10 fold cross validation = {0} for m = {1}".format(rcll_train/len(folds), m))

    print("Average validation accuracy after 10 fold cross validation = {0} for m = {1}".format(accu_valid/len(folds), m))
    print("Average validation precision after 10 fold cross validation = {0} for m = {1}".format(prec_valid/len(folds), m))
    print("Average validation recall after 10 fold cross validation = {0} for m = {1}\n".format(rcll_valid/len(folds), m))

Average training accuracy after 10 fold cross validation = 0.9696144194277926 for m = 3
Average training precision after 10 fold cross validation = 0.910236470415929 for m = 3
Average training recall after 10 fold cross validation = 0.8102257097935016 for m = 3
Average validation accuracy after 10 fold cross validation = 0.8700222660028347 for m = 3
Average validation precision after 10 fold cross validation = 0.3922256858740375 for m = 3
Average validation recall after 10 fold cross validation = 0.2786918852015162 for m = 3

Average training accuracy after 10 fold cross validation = 0.9648438924490443 for m = 4
Average training precision after 10 fold cross validation = 0.9642927636662406 for m = 4
Average training recall after 10 fold cross validation = 0.7143998515339633 for m = 4
Average validation accuracy after 10 fold cross validation = 0.8848742262438248 for m = 4
Average validation precision after 10 fold cross validation = 0.4769901009550452 for m = 4
Average validation recal

### Attempt 3: AdaBoost with multiple weak classifiers and 10 fold cross validation

In [10]:
for t in [10, 25, 50, 100, 200, 250, 500, 1000]:
    rnd_frst = AdaBoostClassifier(n_estimators = t)
    accu_train = prec_train = rcll_train = 0.0
    accu_valid = prec_valid = rcll_valid = 0.0
    for i in range(0, len(folds)):
        rnd_frst.fit(train_input[folds[i][0]], train_output[folds[i][0]])

        train_pred = rnd_frst.predict(train_input[folds[i][0]])
        valid_pred = rnd_frst.predict(train_input[folds[i][1]])

        accu_train += accuracy_score(train_output[folds[i][0]], train_pred)
        prec_train += precision_score(train_output[folds[i][0]], train_pred)
        rcll_train += recall_score(train_output[folds[i][0]], train_pred)
    
        accu_valid += accuracy_score(train_output[folds[i][1]], valid_pred)
        prec_valid += precision_score(train_output[folds[i][1]], valid_pred)
        rcll_valid += recall_score(train_output[folds[i][1]], valid_pred)
    
    print("Average training accuracy after 10 fold cross validation = {0} for t = {1}".format(accu_train/len(folds), t))
    print("Average training precision after 10 fold cross validation = {0} for t = {1}".format(prec_train/len(folds), t))
    print("Average training recall after 10 fold cross validation = {0} for t = {1}".format(rcll_train/len(folds), t))

    print("Average validation accuracy after 10 fold cross validation = {0} for t = {1}".format(accu_valid/len(folds), t))
    print("Average validation precision after 10 fold cross validation = {0} for t = {1}".format(prec_valid/len(folds), t))
    print("Average validation recall after 10 fold cross validation = {0} for t = {1}\n".format(rcll_valid/len(folds), t))

Average training accuracy after 10 fold cross validation = 0.8987311633843629 for t = 10
Average training precision after 10 fold cross validation = 0.6810567932837791 for t = 10
Average training recall after 10 fold cross validation = 0.19101281873603154 for t = 10
Average validation accuracy after 10 fold cross validation = 0.8981063441803887 for t = 10
Average validation precision after 10 fold cross validation = 0.6750924100207478 for t = 10
Average validation recall after 10 fold cross validation = 0.18533689177690107 for t = 10

Average training accuracy after 10 fold cross validation = 0.8987716245411631 for t = 25
Average training precision after 10 fold cross validation = 0.6588623873064288 for t = 25
Average training recall after 10 fold cross validation = 0.21120654142632533 for t = 25
Average validation accuracy after 10 fold cross validation = 0.8979040951395874 for t = 25
Average validation precision after 10 fold cross validation = 0.6489927663002939 for t = 25
Average v

Now it seems that we are randomly trying to use multiple classifiers. I believe that there is a fundamental problem in the preprocessing. I think we have to use the `OneHotEncoder` to encode categorical data after encoding using label as shown [here](http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features).

In [11]:
ohe = OneHotEncoder()
ohe.fit(train_text_vectors)
train_text_ohe_vectors = ohe.transform(train_text_vectors).toarray()
print(train_text_ohe_vectors[0])

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.
  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  1.  0.]


In [13]:
train_input = np.hstack((np.array(train_input_raw)[:,0:1], 
                         train_text_ohe_vectors[:,0:train_text_ohe_vectors.shape[1]], 
                         np.array(train_input_raw)[:,10:13],
                         np.array(train_input_raw)[:,14:]
                        )).astype(float)
train_input = mean_reduce.fit_transform(train_input)
print(train_input.shape)
print('Vector = {0}: \nNorm = {1}'.format(train_input[0], np.linalg.norm(train_input[0])))

train_input = normalize(train_input)
print(train_input.shape)
print('Vector = {0}: \nNorm = {1}'.format(train_input[0], np.linalg.norm(train_input[0])))

folds = []
for tr_split, va_split in skf.split(train_input, train_output):
    folds.append((tr_split, va_split))
for i in range(0, len(folds)):
    print("Fold {0}\tTrain number: {1} Validation number: {2}".format(i+1, len(folds[i][0]), len(folds[i][1])))

(24712, 72)
Vector = [-1.1518788  -0.00636143  1.74069139 -0.54136613 -0.19193262 -0.1629241
 -0.27809875 -0.2094986  -0.19022619 -0.3265319  -0.14790156 -0.44416981
 -0.15868679 -0.08941353 -0.00636143 -0.35667631 -1.23783144  1.60282103
 -0.04271182 -0.00636143 -0.33630791 -0.24338659 -0.41762792 -0.5454267
 -0.02110275 -0.38186992  1.54561294 -0.20897184 -0.00636143  0.51196112
 -0.51183521 -0.00636143 -0.00636143 -0.90828189 -0.15530407  0.9524098
 -0.00636143  0.45871908 -0.15530407 -0.42049726 -0.00636143  0.75750396
 -0.75743781 -0.00636143 -0.26363944  2.37437524 -0.06686689 -0.46226923
 -0.38284321 -0.11544164 -0.70863102 -0.33038794 -0.12794206 -0.11916424
 -0.00636143 -0.48570462  1.93191265 -0.51366037 -0.49186773 -0.49091582
 -0.00636143 -0.34164991  0.40016662 -0.1860759  -0.56425354  0.19653049
 -0.35249416 -1.90310477 -2.38261316  1.97762226 -1.5909553  -1.25826956]: 
Norm = 6.788336172380378
(24712, 72)
Vector = [-0.169685   -0.00093711  0.25642386 -0.07974946 -0.02827

### Attempt 4: SVM with Gaussian Kernel with 10 fold cross validation on modified dataset

In [14]:
accu_train = prec_train = rcll_train = auc_train = 0.0
accu_valid = prec_valid = rcll_valid = auc_valid = 0.0
g_svm = SVC(kernel='rbf', cache_size=2000)
for i in range(0, len(folds)):
    print("Performing fold {0}".format(i+1))
    g_svm.fit(train_input[folds[i][0]], train_output[folds[i][0]])

    train_pred = g_svm.predict(train_input[folds[i][0]])
    valid_pred = g_svm.predict(train_input[folds[i][1]])

    accu_train += accuracy_score(train_output[folds[i][0]], train_pred)
    prec_train += precision_score(train_output[folds[i][0]], train_pred)
    rcll_train += recall_score(train_output[folds[i][0]], train_pred)
    auc_train += roc_auc_score(train_output[folds[i][0]], train_pred)
    
    accu_valid += accuracy_score(train_output[folds[i][1]], valid_pred)
    prec_valid += precision_score(train_output[folds[i][1]], valid_pred)
    rcll_valid += recall_score(train_output[folds[i][1]], valid_pred)
    auc_valid += roc_auc_score(train_output[folds[i][1]], valid_pred)
    
print("Average training accuracy after 10 fold cross validation = {0}".format(accu_train/len(folds)))
print("Average training precision after 10 fold cross validation = {0}".format(prec_train/len(folds)))
print("Average training recall after 10 fold cross validation = {0}".format(rcll_train/len(folds)))
print("Average training AUC-ROC after 10 fold cross validation = {0}".format(auc_train/len(folds)))

print("Average validation accuracy after 10 fold cross validation = {0}".format(accu_valid/len(folds)))
print("Average validation precision after 10 fold cross validation = {0}".format(prec_valid/len(folds)))
print("Average validation recall after 10 fold cross validation = {0}".format(rcll_valid/len(folds)))
print("Average validation AUC-ROC after 10 fold cross validation = {0}".format(auc_valid/len(folds)))

Performing fold 1
Performing fold 2
Performing fold 3
Performing fold 4
Performing fold 5
Performing fold 6
Performing fold 7
Performing fold 8
Performing fold 9
Performing fold 10
Average training accuracy after 10 fold cross validation = 0.8958310813750568
Average training precision after 10 fold cross validation = 0.6276384745715521
Average training recall after 10 fold cross validation = 0.18522490533697172
Average training AUC-ROC after 10 fold cross validation = 0.5856376058714079
Average validation accuracy after 10 fold cross validation = 0.8957588377629966
Average validation precision after 10 fold cross validation = 0.6243785231528323
Average validation recall after 10 fold cross validation = 0.18497073309094658
Average validation AUC-ROC after 10 fold cross validation = 0.5854851984592266


### Attempt 5: Random Forests with multiple number of trees and 10 fold cross validation and modified dataset

In [ ]:
for m in range(3, 9):
    rnd_frst = RandomForestClassifier(n_estimators = m)
    accu_train = prec_train = rcll_train = auc_train = 0.0
    accu_valid = prec_valid = rcll_valid = auc_valid = 0.0
    for i in range(0, len(folds)):
        rnd_frst.fit(train_input[folds[i][0]], train_output[folds[i][0]])

        train_pred = rnd_frst.predict(train_input[folds[i][0]])
        valid_pred = rnd_frst.predict(train_input[folds[i][1]])

        accu_train += accuracy_score(train_output[folds[i][0]], train_pred)
        prec_train += precision_score(train_output[folds[i][0]], train_pred)
        rcll_train += recall_score(train_output[folds[i][0]], train_pred)
        auc_train += roc_auc_score(train_output[folds[i][0]], train_pred)
    
        accu_valid += accuracy_score(train_output[folds[i][1]], valid_pred)
        prec_valid += precision_score(train_output[folds[i][1]], valid_pred)
        rcll_valid += recall_score(train_output[folds[i][1]], valid_pred)
        auc_valid += roc_auc_score(train_output[folds[i][1]], valid_pred)
    
    print("Average training accuracy after 10 fold cross validation = {0} for m = {1}".format(accu_train/len(folds), m))
    print("Average training precision after 10 fold cross validation = {0} for m = {1}".format(prec_train/len(folds), m))
    print("Average training recall after 10 fold cross validation = {0} for m = {1}\n".format(rcll_train/len(folds), m))
    print("Average training AUC-ROC after 10 fold cross validation = {0} for m = {1}\n".format(auc_train/len(folds), m))

    print("Average validation accuracy after 10 fold cross validation = {0} for m = {1}".format(accu_valid/len(folds), m))
    print("Average validation precision after 10 fold cross validation = {0} for m = {1}".format(prec_valid/len(folds), m))
    print("Average validation recall after 10 fold cross validation = {0} for m = {1}\n".format(rcll_valid/len(folds), m))
    print("Average validation AUC-ROC after 10 fold cross validation = {0} for m = {1}\n".format(auc_valid/len(folds), m))    

Average training accuracy after 10 fold cross validation = 0.9694885181258893 for m = 3
Average training precision after 10 fold cross validation = 0.9096471722958366 for m = 3
Average training recall after 10 fold cross validation = 0.8095869553789468 for m = 3

Average training AUC-ROC after 10 fold cross validation = 0.8996883887161916 for m = 3

Average validation accuracy after 10 fold cross validation = 0.8698199186424318 for m = 3
Average validation precision after 10 fold cross validation = 0.3924645475716879 for m = 3
Average validation recall after 10 fold cross validation = 0.28087594440576574 for m = 3

Average validation AUC-ROC after 10 fold cross validation = 0.6127336038743258 for m = 3

Average training accuracy after 10 fold cross validation = 0.9638771925156228 for m = 4
Average training precision after 10 fold cross validation = 0.9646295432446712 for m = 4
Average training recall after 10 fold cross validation = 0.7052197122116501 for m = 4

Average training AUC-RO

### Attempt 6: AdaBoost with multiple weak classifiers and 10 fold cross validation

In [ ]:
for t in [10, 25, 50, 100, 200, 250, 500, 1000]:
    rnd_frst = RandomForestClassifier(n_estimators = t)
    accu_train = prec_train = rcll_train = auc_train = 0.0
    accu_valid = prec_valid = rcll_valid = auc_valid = 0.0
    for i in range(0, len(folds)):
        rnd_frst.fit(train_input[folds[i][0]], train_output[folds[i][0]])

        train_pred = rnd_frst.predict(train_input[folds[i][0]])
        valid_pred = rnd_frst.predict(train_input[folds[i][1]])

        accu_train += accuracy_score(train_output[folds[i][0]], train_pred)
        prec_train += precision_score(train_output[folds[i][0]], train_pred)
        rcll_train += recall_score(train_output[folds[i][0]], train_pred)
        auc_train += roc_auc_score(train_output[folds[i][0]], train_pred)
    
        accu_valid += accuracy_score(train_output[folds[i][1]], valid_pred)
        prec_valid += precision_score(train_output[folds[i][1]], valid_pred)
        rcll_valid += recall_score(train_output[folds[i][1]], valid_pred)
        auc_valid += roc_auc_score(train_output[folds[i][1]], valid_pred)
    
    print("Average training accuracy after 10 fold cross validation = {0} for t = {1}".format(accu_train/len(folds), t))
    print("Average training precision after 10 fold cross validation = {0} for t = {1}".format(prec_train/len(folds), t))
    print("Average training recall after 10 fold cross validation = {0} for t = {1}\n".format(rcll_train/len(folds), t))
    print("Average training AUC-ROC after 10 fold cross validation = {0} for t = {1}\n".format(auc_train/len(folds), t))

    print("Average validation accuracy after 10 fold cross validation = {0} for t = {1}".format(accu_valid/len(folds), t))
    print("Average validation precision after 10 fold cross validation = {0} for t = {1}".format(prec_valid/len(folds), t))
    print("Average validation recall after 10 fold cross validation = {0} for t = {1}\n".format(rcll_valid/len(folds), t))
    print("Average validation AUC-ROC after 10 fold cross validation = {0} for t = {1}\n".format(auc_valid/len(folds), t))    

Average training accuracy after 10 fold cross validation = 0.9826580070722413 for t = 10
Average training precision after 10 fold cross validation = 0.9823910636246657 for t = 10
Average training recall after 10 fold cross validation = 0.8615107852929416 for t = 10

Average training AUC-ROC after 10 fold cross validation = 0.9297749117888896 for t = 10

Average validation accuracy after 10 fold cross validation = 0.8853998214382417 for t = 10
Average validation precision after 10 fold cross validation = 0.48157310542870624 for t = 10
Average validation recall after 10 fold cross validation = 0.21982671926974548 for t = 10

Average validation AUC-ROC after 10 fold cross validation = 0.5948641382403228 for t = 10

Average training accuracy after 10 fold cross validation = 0.993678287210772 for t = 25
Average training precision after 10 fold cross validation = 0.9906729963843015 for t = 25
Average training recall after 10 fold cross validation = 0.952865681247242 for t = 25

Average train